# CMAP-Elites DEMO

## Imports

In [ ]:
import json

GECCO-compatible `matplotlib` options:

In [ ]:
import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

Import `PCGSEPy` modules:

In [ ]:
from pcgsepy.common.vecs import orientation_from_str, Vec 
from pcgsepy.config import COMMON_ATOMS, HL_ATOMS, N_ITERATIONS, REQ_TILES 
from pcgsepy.lsystem.rules import RuleMaker
from pcgsepy.lsystem.actions import AtomAction, Rotations
from pcgsepy.lsystem.parser import HLParser, LLParser
from pcgsepy.lsystem.solver import LSolver
from pcgsepy.lsystem.constraints import ConstraintHandler, ConstraintLevel, ConstraintTime
from pcgsepy.lsystem.constraints_funcs import components_constraint, intersection_constraint, symmetry_constraint, axis_constraint
from pcgsepy.lsystem.lsystem import LSystem
from pcgsepy.structure import block_definitions
from pcgsepy.evo.genops import expander

## Setup

In [ ]:
with open(COMMON_ATOMS, "r") as f:
    common_alphabet = json.load(f)

for k in common_alphabet:
    action, args = common_alphabet[k]["action"], common_alphabet[k]["args"]
    action = AtomAction(action)
    if action == AtomAction.MOVE:
        args = orientation_from_str[args]
    elif action == AtomAction.ROTATE:
        args = Rotations(args)
    common_alphabet[k] = {"action": action, "args": args}

In [ ]:
with open(HL_ATOMS, "r") as f:
    hl_atoms = json.load(f)

tiles_dimensions = {}
tiles_block_offset = {}
for tile in hl_atoms.keys():
    dx, dy, dz = hl_atoms[tile]["dimensions"]
    tiles_dimensions[tile] = Vec.v3i(dx, dy, dz)
    tiles_block_offset[tile] = hl_atoms[tile]["offset"]

hl_alphabet = {}
for k in common_alphabet.keys():
    hl_alphabet[k] = common_alphabet[k]

for hk in hl_atoms.keys():
    hl_alphabet[hk] = {"action": AtomAction.PLACE, "args": []}

In [ ]:
ll_alphabet = {}

for k in common_alphabet.keys():
    ll_alphabet[k] = common_alphabet[k]

# for k in block_definitions.keys():
#     if k != "":  # TODO: This is a probable bug, reported to the SE API devs
#         ll_alphabet[k] = {"action": AtomAction.PLACE, "args": [k]}

In [ ]:
used_ll_blocks = [
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCorner',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorSlope',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorCornerInv',
    'MyObjectBuilder_CubeBlock_LargeBlockArmorBlock',
    'MyObjectBuilder_Gyro_LargeBlockGyro',
    'MyObjectBuilder_Reactor_LargeBlockSmallGenerator',
    'MyObjectBuilder_CargoContainer_LargeBlockSmallContainer',
    'MyObjectBuilder_Cockpit_OpenCockpitLarge',
    'MyObjectBuilder_Thrust_LargeBlockSmallThrust',
    'MyObjectBuilder_InteriorLight_SmallLight',
    'MyObjectBuilder_CubeBlock_Window1x1Slope',
    'MyObjectBuilder_CubeBlock_Window1x1Flat',
    'MyObjectBuilder_InteriorLight_LargeBlockLight_1corner'
]

for k in used_ll_blocks:
    ll_alphabet[k] = {"action": AtomAction.PLACE, "args": [k]}

## L-System components

In [ ]:
hl_rules = RuleMaker(ruleset='hlrules').get_rules()
ll_rules = RuleMaker(ruleset='llrules').get_rules()

hl_parser = HLParser(rules=hl_rules)
ll_parser = LLParser(rules=ll_rules)

hl_solver = LSolver(parser=hl_parser,
                    atoms_alphabet=hl_alphabet,
                    extra_args={
                        'tiles_dimensions': tiles_dimensions,
                        'tiles_block_offset': tiles_block_offset,
                        'll_rules': ll_rules
                    })
ll_solver = LSolver(parser=ll_parser,
                    atoms_alphabet=dict(hl_alphabet, **ll_alphabet),
                    extra_args={})

In [ ]:
rcc1 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc1.extra_args["req_tiles"] = ['cockpit']

rcc2 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc2.extra_args["req_tiles"] = ['corridorcargo', 'corridorgyros', 'corridorreactors']

rcc3 = ConstraintHandler(
    name="required_components",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.END,
    f=components_constraint,
    extra_args={
        'alphabet': hl_alphabet
    }
)
rcc3.extra_args["req_tiles"] = ['thrusters']

nic = ConstraintHandler(
    name="no_intersections",
    level=ConstraintLevel.HARD_CONSTRAINT,
    when=ConstraintTime.DURING,
    f=intersection_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    },
    needs_ll=True
)
nic.extra_args["tiles_dimensions"] = tiles_dimensions

sc = ConstraintHandler(
    name="symmetry",
    level=ConstraintLevel.SOFT_CONSTRAINT,
    when=ConstraintTime.END,
    f=symmetry_constraint,
    extra_args={
        'alphabet': dict(hl_alphabet, **ll_alphabet)
    }
)

In [ ]:
lsystem = LSystem(
    hl_solver=hl_solver, ll_solver=ll_solver, names=['HeadModule', 'BodyModule', 'TailModule']
)

In [ ]:
lsystem.add_hl_constraints(cs=[
    [nic, rcc1],
    [nic, rcc2],
    [nic, rcc3]
])

lsystem.add_ll_constraints(cs=[
    [sc],
    [sc],
    [sc]
])

In [ ]:
expander.initialize(rules=lsystem.hl_solver.parser.rules)

## MAP-Elites

In [ ]:
from pcgsepy.evo.fitness import box_filling_fitness, bounding_box_fitness, func_blocks_fitness, axis_fitness

feasible_fitnesses = [bounding_box_fitness,
                      box_filling_fitness,
                      func_blocks_fitness,
                      axis_fitness]

In [ ]:
from pcgsepy.mapelites.behaviors import BehaviorCharacterization, mame, mami

b0 = BehaviorCharacterization(name='Major axis / Medium axis',
                              func=mame,
                              bounds=(0, 20))
b1 = BehaviorCharacterization(name='Major axis / Smallest axis',
                              func=mami,
                              bounds=(0, 20))

In [ ]:
from pcgsepy.mapelites.map import MAPElites

In [ ]:
mapelites = MAPElites(lsystem=lsystem,
                      feasible_fitnesses=feasible_fitnesses,
                      behavior_descriptors=(b0, b1),
                      n_bins=(8, 8))

In [ ]:
# mapelites.generate_initial_populations(pops_size=20,
#                                        n_retries=100)

# mapelites.show_fitness(show_mean=True,
#                        population='feasible')
# mapelites.show_fitness(show_mean=False,
#                        population='feasible')
# mapelites.show_fitness(show_mean=True,
#                        population='infeasible')
# mapelites.show_fitness(show_mean=False,
#                        population='infeasible')
# mapelites.show_coverage(population='feasible')
# mapelites.show_coverage(population='infeasible')
# mapelites.show_age(show_mean=True,
#                    population='feasible')
# mapelites.show_age(show_mean=True,
#                    population='infeasible')
# mapelites.show_age(show_mean=False,
#                    population='feasible')
# mapelites.show_age(show_mean=False,
#                    population='infeasible')

In [ ]:
# from tqdm.notebook import trange

# for i in trange(2):
#     mapelites.rand_step(gen=i)

# # mapelites.interactive_mode(n_steps=5)

# mapelites.show_fitness(show_mean=True,
#                        population='feasible')
# mapelites.show_fitness(show_mean=False,
#                        population='feasible')
# mapelites.show_fitness(show_mean=True,
#                        population='infeasible')
# mapelites.show_fitness(show_mean=False,
#                        population='infeasible')
# mapelites.show_coverage(population='feasible')
# mapelites.show_coverage(population='infeasible')
# mapelites.show_age(show_mean=True,
#                    population='feasible')
# mapelites.show_age(show_mean=True,
#                    population='infeasible')
# mapelites.show_age(show_mean=False,
#                    population='feasible')
# mapelites.show_age(show_mean=False,
#                    population='infeasible')

## DASH

In [ ]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from dash.exceptions import PreventUpdate
import logging
from datetime import datetime

import matplotlib.pyplot as plt

from typing import Tuple

In [ ]:
class DashLoggerHandler(logging.StreamHandler):
    def __init__(self):
        logging.StreamHandler.__init__(self)
        self.queue = []

    def emit(self, record):
        t = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        msg = self.format(record)
        self.queue.append(f'[{t}]\t{msg}')

logger = logging.getLogger('dash-msgs')
logger.setLevel(logging.DEBUG)
dashLoggerHandler = DashLoggerHandler()
logger.addHandler(dashLoggerHandler)

In [ ]:
from pcgsepy.config import BIN_POP_SIZE, CS_MAX_AGE

hm_callback_props = {
    'pop': {
        'Feasible': 'feasible',
        'Infeasible': 'infeasible'
    },
    'metric': {
        'Fitness': {
            'name': 'fitness',
            'zmax': 4.5,  # TODO: get from mapelites
            'colorscale': 'Inferno'
        },
        'Age':  {
            'name': 'age',
            'zmax': CS_MAX_AGE,
            'colorscale': 'Greys'
        },
        'Coverage': {
            'name': 'size',
            'zmax': BIN_POP_SIZE,
            'colorscale': 'Hot'
        }
    },
    'method': {
        'Population': True,
        'Elitist': False
    }
}

In [ ]:
def from_bc_to_idx(bcs: Tuple[float, float], me: MAPElites) -> Tuple[int, int]:
    b0, b1 = bcs
    return (int(b0 // me.bin_sizes[0]), int(b1 // me.bin_sizes[1]))


gen_counter = 0

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__,
                title='SE ICMAP-Elites',
                external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Div([
        html.H1(children="Space Engineers ICMAP-Elites", style={'textAlign': 'center'}),
        html.Div([
            html.H6('Plot settings', style={'textAlign': 'center'}),
            html.P('Choose which population to display.'),
            dcc.Dropdown(
                ['Feasible', 'Infeasible'],
                'Feasible',
                id='population-dropdown',
            ),
            html.P('Choose which metric to plot.'),
            dcc.Dropdown(
                ['Fitness', 'Age', 'Coverage'],
                'Fitness',
                id='metric-dropdown',
            ),
            html.P('Choose whether to compute the metric for the entire bin population or just the elitist.'),
            dcc.RadioItems(
                ['Population', 'Elitist'],
                'Population',
                id='method-radio',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )], style={'width': '15%', 'display': 'inline-block', 'verticalAlign': 'top'}),
        html.Div([
            dcc.Graph(id="heatmap-plot", style={'width': '100%', 'height': '100%'})
        ], style={'width': '40%', 'height': '100%', 'display': 'inline-block', 'verticalAlign': 'top'}),
        html.Div([
            dcc.Graph(id="content-plot", style={'width': '100%', 'height': '100%'})
        ], style={'width': '30%', 'height': '100%', 'display': 'inline-block', 'verticalAlign': 'top'}),
        html.Div([
            html.H6('Experiment settings', style={'textAlign': 'center'}),
            html.P('Valid bins are: ', id='valid-bins'),
            html.P(f'Current generation: {gen_counter}', id='gen-display'),
            html.P('Selected bin(s): []', id='selected-bin'),
            html.P('', id='hidden-p', style={'display': 'none'}),
            html.H6('Experiment controls', style={'textAlign': 'center'}),
            html.Button('Initialize/Reset', id='reset-btn', n_clicks=0, style={'horizontalAlign': 'center'}),
            html.Br(),
            html.Button('Toggle single bin selection', id='selection-btn', n_clicks=0, style={'horizontalAlign': 'center'}),
            html.Button('Clear selection', id='selection-clr-btn', n_clicks=0, style={'horizontalAlign': 'center'}),
            html.Br(),
            html.Button('Apply step', id='step-btn', n_clicks=0, style={'horizontalAlign': 'center'})
        ], style={'width': '15%', 'display': 'inline-block', 'verticalAlign': 'top', 'horizontalAlign': 'right'})
    ]),
    html.Br(),
    html.Div([
        dcc.Interval(id='interval1', interval=1 * 1000, n_intervals=0),
        html.P(id='div-out', children='Log', style={'textAlign': 'center'}),
        html.Iframe(id='console-out', srcDoc='', style={'width': '100%'})
    ], style={'width': '30%', 'height': 200})
])

In [ ]:
def switch(ls):
    res = []
    for e in ls:
        res.append((e[1], e[0]))
    return res


def get_valid_bins():
    valid_bins = [x.bin_idx for x in mapelites._valid_bins()]
    return switch(valid_bins)


def build_heatmap(pop_name,
                  metric_name,
                  method_name):
    metric = hm_callback_props['metric'][metric_name]
    use_mean = hm_callback_props['method'][method_name]
    population = hm_callback_props['pop'][pop_name]
    # build hotmap
    disp_map = np.zeros(shape=mapelites.bins.shape)
    for i in range(mapelites.bins.shape[0]):
        for j in range(mapelites.bins.shape[1]):
            disp_map[i, j] = mapelites.bins[i, j].get_metric(metric=metric['name'],
                                                             use_mean=use_mean,
                                                             population=population)
    # plot
    x_labels = np.arange(0, mapelites.limits[0], mapelites.bin_sizes[0])
    y_labels = np.arange(0, mapelites.limits[1], mapelites.bin_sizes[1])
    title = f'{pop_name} population {metric_name.lower()} ({"Average" if use_mean else "Elitist"})'
    heatmap = px.imshow(disp_map,
                        zmin=0,
                        zmax=hm_callback_props['metric'][metric_name]['zmax'],
                        origin='lower',
                        labels=dict(x=mapelites.b_descs[0].name,
                                    y=mapelites.b_descs[1].name,
                                    color=metric_name),
                        x=x_labels,
                        y=y_labels,
                        title=title,
                        aspect='equal',
                        color_continuous_scale=hm_callback_props['metric'][metric_name]['colorscale'])
    heatmap.update_layout(clickmode='event+select')
    heatmap.update_layout(
        xaxis={
            'tickmode': 'linear',
            'tick0': 0,
            'dtick': mapelites.bin_sizes[0]
        },
        yaxis={
            'tickmode': 'linear',
            'tick0': 0,
            'dtick': mapelites.bin_sizes[1]
        }
    )
    return heatmap


def get_elite_content(me: MAPElites,
                      bin_idx,
                      pop):
    # get elitist content
    elite = me.get_elitist(bin_idx=bin_idx,
                           pop=pop).content
    structure = elite.as_array()
    arr = np.nonzero(structure)
    x, y, z = arr
    cs = [structure[i, j, k] for i, j, k in zip(x, y, z)]
    ss = [elite._clean_label(elite.ks[v]) for v in cs]
    fig = px.scatter_3d(x=x,
                        y=y,
                        z=z,
                        color=ss,
                        symbol=['square' for _ in range(len(x))],
                        labels={
                            'x': 'x',
                            'y': 'y',
                            'z': 'z',
                            'color': 'Block type'
                        },
                        title='Elite content')
    fig.update_layout(scene=dict(aspectmode='data'))
    return fig

In [ ]:
@app.callback(
    Output('console-out', 'srcDoc'),
    Input('interval1', 'n_intervals'))
def update_output(n):
    return ('\n'.join(dashLoggerHandler.queue)).replace('\n', '<BR>')


@app.callback(Output('hidden-p', 'children'),
              Output('selected-bin', 'children'),
              Output('content-plot', 'figure'),
              Input('heatmap-plot', 'clickData'),
              Input('population-dropdown', 'value'),
              Input('selection-btn', 'n_clicks'),
              Input('selection-clr-btn', 'n_clicks'),
              State('selected-bin', 'children'),
              State('content-plot', 'figure'))
def display_click_data(clickData,
                       pop_name,
                       selection_btn,
                       clear_btn,
                       curr_selected,
                       curr_fig):
    ctx = dash.callback_context

    if not ctx.triggered:
        event_trig = None
    else:
        event_trig = ctx.triggered[0]['prop_id'].split('.')[0]

    if event_trig == 'heatmap-plot' or event_trig == 'population_dropdown':
        i, j = from_bc_to_idx(bcs=(clickData['points'][0]['x'],
                                   clickData['points'][0]['y']),
                              me=mapelites)
        if mapelites.non_empty(bin_idx=(j, i),
                               pop='feasible' if pop_name == 'Feasible' else 'infeasible'):
            fig = get_elite_content(me=mapelites,
                                    bin_idx=(j, i),
                                    pop='feasible' if pop_name == 'Feasible' else 'infeasible')
            if not mapelites.enforce_qnt and curr_selected != 'Selected bin(s): []':
                curr_selected = curr_selected.split(':')[1].split(';')
                curr_selected = [(int(x.split(',')[0].replace('(', '')), int(x.split(',')[1].replace(')', ''))) for x in curr_selected]
                curr_selected.append((i, j))
            else:
                curr_selected = [(i, j)]
            return '; '.join([str(x) for x in curr_selected]), f'Selected bin(s): {"; ".join([str(x) for x in switch(curr_selected)])}', fig
        else:
            logging.getLogger('dash-msgs').error(msg=f'Empty bin selected ({i}, {j}).')
            raise PreventUpdate
    elif event_trig == 'selection-btn':
        mapelites.enforce_qnt = not mapelites.enforce_qnt
        logging.getLogger('dash-msgs').debug(msg=f'MAP-Elites single bin selection set to {mapelites.enforce_qnt}.')
        if mapelites.enforce_qnt:
            curr_selected = curr_selected.split(':')[1].split(';')
            curr_selected = [(int(x.split(',')[0].replace('(', '')), int(x.split(',')[1].replace(')', ''))) for x in curr_selected]
            curr_selected = [curr_selected[-1]]
        return '; '.join([str(x) for x in curr_selected]), f'Selected bin(s): {"; ".join([str(x) for x in switch(curr_selected)])}', curr_fig
    elif event_trig == 'selection-clr-btn':
        logging.getLogger('dash-msgs').debug(msg='Cleared bins selection.')
        return '', f'Selected bin(s): {[]}', curr_fig
    else:
        raise PreventUpdate


@app.callback(Output('heatmap-plot', 'figure'),
              Output('valid-bins', 'children'),
              Output('gen-display', 'children'),
              State('hidden-p', 'children'),
              State('heatmap-plot', 'figure'),
              Input('population-dropdown', 'value'),
              Input('metric-dropdown', 'value'),
              Input('method-radio', 'value'),
              Input('step-btn', 'n_clicks'),
              Input('reset-btn', 'n_clicks'),)
def update_heatmap(bin_idxs,
                   curr_heatmap,
                   pop_name,
                   metric_name,
                   method_name,
                   n_clicks_step,
                   n_clicks_reset):
    global gen_counter

    ctx = dash.callback_context

    if not ctx.triggered:
        event_trig = None
    else:
        event_trig = ctx.triggered[0]['prop_id'].split('.')[0]

    if event_trig == 'step-btn':
        if bin_idxs is not None and bin_idxs != '':
            bin_idxs = bin_idxs.split('; ')
            bin_idxs = [(int(x.split(',')[0].replace('(', '')), int(x.split(',')[1].replace(')', ''))) for x in bin_idxs]
            valid = True
            if mapelites.enforce_qnt:
                valid_bins = [x.bin_idx for x in mapelites._valid_bins()]
                for bin_idx in bin_idxs:
                    valid &= bin_idx in valid_bins
            if valid:
                logging.getLogger('dash-msgs').debug(msg=f'Started step {gen_counter}...')
                mapelites._interactive_step(bin_idxs=bin_idxs,
                                            n_gen=gen_counter)
                gen_counter += 1
                curr_heatmap = build_heatmap(pop_name=pop_name,
                                             metric_name=metric_name,
                                             method_name=method_name)
                logging.getLogger('dash-msgs').debug(msg=f'Completed step {gen_counter - 1}.')
            else:
                logging.getLogger('dash-msgs').error(msg=f'Step not applied: invalid bin(s) selected.')
        return curr_heatmap, f'Valid bins are: {get_valid_bins()}', f'Current generation: {gen_counter}'
    elif event_trig == 'reset-btn':
        logging.getLogger('dash-msgs').debug(msg=f'Started resetting all bins...')
        gen_counter = 0
        mapelites.reset()
        heatmap = build_heatmap(pop_name=pop_name,
                                metric_name=metric_name,
                                method_name=method_name)
        logging.getLogger('dash-msgs').debug(msg=f'Reset completed.')
        return heatmap, f'Valid bins are: {get_valid_bins()}', f'Current generation: {gen_counter}'
    else:
        heatmap = build_heatmap(pop_name=pop_name,
                                metric_name=metric_name,
                                method_name=method_name)
        return heatmap, f'Valid bins are: {get_valid_bins()}', f'Current generation: {gen_counter}'

In [ ]:
app.run_server(debug=True, use_reloader=False)